In [1]:
path_to_football_file = "/home/nalimov/Загрузки/football"

# todo: сделать создание 
# ./data
# ./data/filtered_prepared
# ./original_by_time_period

# пути выше, должны существовать

Функция ниже, нарезает данные на 3 части (начало, середина, конец чемпионата)

In [ ]:
import datetime
from datetime import datetime as dt

# full count: 39 789 519

# c 14 июня 2018
# по 15 июля 2018

def write_texts_to_file(filename, texts):
    f = open(filename, 'w')

    for i in texts:
        f.write(i + '\n')

    f.close()

def devide_file_by_periods():
    first = True

    count_all = 0

    datetime_set = {}

    queries_start = []
    queries_middle = []
    queries_end = []

    # todo: по хорошему надо прочитать как csv и pandas-ом отфильтровать по дате

    with open(path_to_football_file) as infile:
        for line in infile:
            if first:
                first = False
                continue

            count_all += 1

            line_splitted = line.replace('\n', '').split('\t')

            query = line_splitted[0]
            datetime_var = line_splitted[-1]

            if not datetime_var in datetime_set:
                datetime_set[datetime_var] = None

            datetime_object = dt.strptime(datetime_var, '%Y-%m-%d %H:%M:%S')

            if datetime_object <= datetime_start_1:
                queries_start.append(query)
            elif datetime_object <= datetime_end_0:
                queries_middle.append(query)
            elif datetime_object <= datetime_end_1:
                queries_end.append(query)
            else:
                print('no one, check you clauses')
                
    print(len(queries_start))
    print(len(queries_middle))
    print(len(queries_end))

    write_texts_to_file('./data/original_by_time_period/queries_start', queries_start)
    write_texts_to_file('./data/original_by_time_period/queries_middle', queries_middle)
    write_texts_to_file('./data/original_by_time_period/queries_end', queries_end)

    datetime_set = sorted(datetime_set.keys())
    print('full: ' + str(count_all))
    print(datetime_set[0], datetime_set[-1])


datetime_start_0 = datetime.datetime(2018, 6, 14)
# start
datetime_start_1 = datetime.datetime(2018, 6, 24)
# middle
datetime_end_0 = datetime.datetime(2018, 7, 5)
# end 1 день для правильного условия
datetime_end_1 = datetime.datetime(2018, 7, 16)

if __name__ == "__main__":
    data = devide_file_by_periods()

Вспомогательные функции для чтения и записи запросов

In [2]:
from nltk.corpus import stopwords

nltk_stop_words = list(stopwords.words('english'))
nltk_stop_words += list(stopwords.words('russian'))


# todo: stemming, lemming
def read_queries(filename, need_split=False, need_remove_stop_words=False, max_count=None):
    queries = []

    num = 0

    with open(filename) as infile:
        for line in infile:
            line = line.replace('\n', '')

            if need_remove_stop_words:
                s_splitted_line = line.split()
                s_splitted_line = [i for i in s_splitted_line if i not in nltk_stop_words]

                if need_split:
                    queries.append(s_splitted_line)
                else:
                    queries.append(' '.join(s_splitted_line))
            else:
                if need_split:
                    queries.append(line.split())
                else:
                    queries.append(line)

            num += 1

            if max_count is not None:
                if max_count == num:
                    break

    print(' ')

    return queries


def write_queries(filename, queries):
    f = open(filename, 'w')

    for i in queries:
        f.write(i + '\n')

    f.close()

Следующая функция оставит только те запросы, в которых присутствуют ключевые слова

In [5]:
words = [
    'чемпионат',
    'чемпионату',
    'чемпионате',

    'мира',

    'футбол',
    'футболу',

    # '2018',
    'чм',
    'матчи',
    'раписание',
]

def filter_prepare(period_type):
    print ("start " + period_type)
    
    queries_start = read_queries('./data/original_by_time_period/' + period_type,
                                 need_split=False, need_remove_stop_words=False)

    print (len(queries_start))

    choosed_queries = []

    num = 0

    for query in queries_start:
        for word in words:
            if word in query:
                choosed_queries.append(query)
                break

        num += 1

    print ("num for: ", period_type, len(choosed_queries))

    write_queries('./data/filtered_prepared/' + period_type, choosed_queries)

filter_prepare('queries_start')
filter_prepare('queries_middle')
filter_prepare('queries_end')


start queries_start
./data/original_by_time_period/queries_start
 
12850460
num for:  queries_start 327027
start queries_middle
./data/original_by_time_period/queries_middle
 
13760275
num for:  queries_middle 338000
start queries_end
./data/original_by_time_period/queries_end
 
13178784
num for:  queries_end 232831
